In [1]:
import numpy as np

In [2]:
from dust_gan import DustDCGAN,DCGAN

Importing necessary packages and modules


d:\my_pythonanaconda3\envs\dust_gan_env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
t1 = DustDCGAN('D:\Projects\Maps_and_Makers\Planck_dust_cuts_353GHz.pk',load_state=True,test=False)

Loading Data
Loading Previous State


In [4]:
t1.adversarial.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_7 (Sequential)    (None, 600, 600, 1)       3431361   
_________________________________________________________________
sequential_8 (Sequential)    (None, 1)                 1691585   
Total params: 5,122,946
Trainable params: 3,379,777
Non-trainable params: 1,743,169
_________________________________________________________________


In [5]:
t1.adversarial.get_layer('sequential_6').summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_7 (Sequential)    (None, 1)                 1691585   
Total params: 1,691,585
Trainable params: 1,690,817
Non-trainable params: 768
_________________________________________________________________


In [9]:
t1.train(train_steps=20, batch_size=32, save_interval=10)


Training Beginning
0: [D loss: 0.244054, acc: 0.937500]  [A loss: 2.540958, acc: 0.031250]
